In [1]:
from keras.layers import Dense,LSTM,Embedding
from keras.models import Sequential,Model
import tensorflow as tf
import keras
import logging
import numpy as np

tf.get_logger().setLevel(logging.ERROR)

np.random.seed(1234)
tf.random.set_seed(1234)

def _one_hot_encoding(x):
    x_categorical = []
    for row in x:
        row_categorical = []
        index = 0
        for feature in row:
            if feature == 'sms':
                row_categorical.append([1,0,0])
            if feature == 'search':
                row_categorical.append([0,1,0])
            if feature == 'email':
                row_categorical.append([0,0,1])
            index += 1
        empty = 3 - index
        for i in range(empty):
            row_categorical.append([0,0,0])
        x_categorical.append(row_categorical)
    return x_categorical


def extract_data_into_X_and_Y(data):
    x = []
    for row in data:
        x.append(row[:-1])
    y = [row[-1] for row in data]
    x = _one_hot_encoding(x)
    return x, y

def flatten_X(x):
    x_flat = []
    for row in x:
        row_flat = []
        for vector in row:
            for value in vector:
                row_flat.append(value)
        x_flat.append(row_flat)
    return x_flat

def create_model_LSTM():
    model = Sequential()
    model.add(LSTM(18, input_shape=(3, 3)))
    model.add(Dense(9, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc'])
    return model
    
def create_model_Dense():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(9,)))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc'])
    return model

In [2]:
data = (
    ('sms', 'search', 'email', 1),
    ('search', 'sms', 0),
    ('search', 0),
    ('sms', 'search', 'sms', 0),
    ('email', 'sms', 1),
    ('sms', 0),
    ('email', 1),
    ('email', 'email', 1),
    ('sms', 'sms', 0),
    ('email', 'email', 'email', 1),
    ('sms', 'sms', 0),
    ('search', 0),
    ('sms', 'search', 'sms', 0),
    ('email', 'search', 1),
    ('sms', 0),
    ('email', 1),
    ('email', 'email', 1),
    ('sms', 'sms', 0)
)

x, y = extract_data_into_X_and_Y(data)
x_flat = flatten_X(x)

repeats = 10
print('LSTM')
for i in range(repeats):
    model_lstm = create_model_LSTM()
    model_lstm.fit(x, y, epochs=10, batch_size=10, shuffle=False, verbose=0)
    model_lstm.evaluate(x,y)
   
print('Dense')
for i in range(repeats):
    model_dense = create_model_Dense()
    model_dense.fit(x_flat, y, epochs=10, verbose=0)
    model_dense.evaluate(x_flat,y)

LSTM
1/1 [==============================] - 0s 1ms/step - loss: 5.2982e-08 - acc: 0.2778
Dense
1/1 [==============================] - 0s 1ms/step - loss: 5.2982e-08 - acc: 0.3333


In [3]:
repeats = 10
print('LSTM')
for i in range(repeats):
    model_lstm = create_model_LSTM()
    model_lstm.fit(x, y, epochs=2, batch_size=18, verbose=0)
    model_lstm.evaluate(x,y)
   
print('Dense')
for i in range(repeats):
    model_dense = create_model_Dense()
    model_dense.fit(x_flat, y, epochs=2, verbose=0)
    model_dense.evaluate(x_flat,y)

LSTM
1/1 [==============================] - 0s 1ms/step - loss: 5.2982e-08 - acc: 0.2222
Dense
1/1 [==============================] - 0s 1ms/step - loss: 5.2982e-08 - acc: 0.9444
